In [1]:
import pandas as pd

In [9]:
df = pd.read_csv('climate_price_merged_data.csv').set_index('Unnamed: 0').reset_index(drop = True)
print(df.shape)
df.head()

(3267, 24)


,state,county_risk,population,area,risk_value,risk_score,risk_ratng,county_match,county_price,price,...,black_pct,asian_pct,hispanic_pct,ed_pct,county_tax,tax_paid_2022,tax_trend,state_name,county_unemp,unemployment_rate
0,AK,Aleutians East,3374.0,15167.694230,8.949653e+05,4.231626,Very Low,aleutianseast,NaN,NaN,...,0.086842,0.225439,0.197076,15.935335,NaN,NaN,NaN,alaska,Aleutians East,1.4
1,AK,Aleutians West,5168.0,14258.993100,4.151810e+06,35.221126,Very Low,aleutianswest,NaN,NaN,...,0.049121,0.287080,0.127676,16.805628,NaN,NaN,NaN,alaska,Aleutians West,1.8
2,AK,Anchorage,290985.0,1966.338483,9.456571e+07,94.845689,Relatively Moderate,anchorage,Anchorage,373377.911057,...,0.047303,0.093670,0.090775,36.772161,NaN,NaN,NaN,alaska,Anchorage,3.3
3,AK,Bethel,18633.0,46015.507070,2.354071e+06,16.926503,Very Low,bethel,NaN,NaN,...,0.004500,0.010768,0.011090,12.728180,NaN,NaN,NaN,alaska,Bethel,9.4
4,AK,Bristol Bay,843.0,857.255664,1.055605e+05,0.159084,Very Low,bristolbay,NaN,NaN,...,0.007109,0.005924,0.053318,22.083981,NaN,NaN,NaN,alaska,Bristol Bay,8.2


In [21]:
pd.DataFrame({'column': df.columns, 'null count': [sum(df.isnull()[column]) for column in df.columns]})

,column,null count
0,state,0
1,county_risk,2
2,population,2
3,area,2
4,risk_value,90
5,risk_score,90
6,risk_ratng,2
7,county_match,0
8,county_price,431
9,price,432


Null value counts are fairly low in the data. Attempt to remove null values to see total number of records remaining

In [24]:
print(f'Total Rows before removing nulls: {df.shape[0]}')

Total Rows before removing nulls: 3267


In [38]:
for col in ['county_risk', 'county_price', 'county_inc', 'county_dem', 'county_tax', 'county_unemp', 'price', 'white_pct', 'tax_paid_2022']:
    df.drop(df[df[col].isnull()].index, inplace = True)
    df.reset_index(inplace = True, drop = True)

In [59]:
print(f'Total Rows before removing nulls: {df.shape[0]}')
# Lost 594 rows of data, <20% of the data. This should be fine
df['population_density'] = df['population']/df['area']

Total Rows before removing nulls: 2673


In [61]:
# Predictor/Outcome variables
main = df[['population_density', 'risk_score', 'income', 'white_pct', 'black_pct', 'asian_pct', 'hispanic_pct', 'ed_pct', 
    'tax_paid_2022', 'tax_trend', 'unemployment_rate','price']]

In [63]:
main.describe()

,population_density,risk_score,income,white_pct,black_pct,asian_pct,hispanic_pct,ed_pct,tax_paid_2022,tax_trend,unemployment_rate,price
count,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2.673000e+03
mean,234.490469,52.772359,57026.824542,0.753445,0.082544,0.014419,0.093978,23.351807,1915.897119,8.278451,3.757389,2.646938e+05
std,1368.316965,27.938820,17989.057432,0.186439,0.133410,0.028417,0.126225,9.916170,1410.132422,5.289147,1.442241,1.672879e+05
min,0.239075,0.095450,25013.000000,0.017764,0.000000,0.000000,0.004236,6.384911,256.000000,-37.300000,1.400000,2.986062e+04
25%,21.020022,28.985046,47364.000000,0.648231,0.005535,0.003447,0.024418,16.344561,1017.000000,5.500000,2.900000,1.741580e+05
50%,47.462962,53.706650,53553.000000,0.815741,0.021557,0.005750,0.046953,20.825037,1544.000000,8.000000,3.500000,2.253736e+05
75%,121.134367,76.741966,61974.000000,0.903642,0.091198,0.013055,0.105178,27.952144,2355.000000,10.800000,4.300000,3.025106e+05
max,49520.806441,100.000000,471751.000000,0.969518,0.845455,0.422436,0.976836,76.304063,10001.000000,54.000000,19.800000,2.780321e+06


In [69]:
main.to_csv('climate_nonnull.csv')

# Fit model without risk

In [65]:
formula = 'price ~ population_density + income + white_pct + black_pct + asian_pct + hispanic_pct + ed_pct + tax_paid_2022 + tax_trend + unemployment_rate'

model = sm.GLM.from_formula(formula, data=main, family=sm.families.Gaussian())
result = model.fit()

result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  price   No. Observations:                 2673
Model:                            GLM   Df Residuals:                     2662
Model Family:                Gaussian   Df Model:                           10
Link Function:               Identity   Scale:                      1.2209e+10
Method:                          IRLS   Log-Likelihood:                -34828.
Date:                Fri, 06 Dec 2024   Deviance:                   3.2501e+13
Time:                        06:29:36   Pearson chi2:                 3.25e+13
No. Iterations:                     3   Pseudo R-squ. (CS):             0.7262
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           2.986e+04   3.71e+04      0.804      0.421   -4.29e+04    1.03e+05
population_density    19.5678      1.700     11.509      0.000      16.235      22.900
income                 4.2759      0.161     26.567      0.000       3.960       4.591
white_pct          -2.006e+05   3.76e+04     -5.341      0.000   -2.74e+05   -1.27e+05
black_pct          -2.695e+05   3.87e+04     -6.969      0.000   -3.45e+05   -1.94e+05
asian_pct           3.142e+05   1.09e+05      2.874      0.004    9.99e+04    5.28e+05
hispanic_pct       -1.074e+05   4.05e+04     -2.649      0.008   -1.87e+05   -2.79e+04
ed_pct              4138.4255    345.169     11.990      0.000    3461.908    4814.943
tax_paid_2022         -0.5289      2.285     -0.231      0.817      -5.007       3.949
tax_trend           1881.4691    409.828      4.591      0.000    1078.222    2684.716
unemployment_rate    1.44e+04   1591.400      9.051      0.000    1.13e+04    1.75e+04
======================================================================================
"""

In [67]:
## Fit model with climate risk
formula = 'price ~ risk_score + population_density + income + white_pct + black_pct + asian_pct + hispanic_pct + ed_pct + tax_paid_2022 + tax_trend + unemployment_rate'

model = sm.GLM.from_formula(formula, data=main, family=sm.families.Gaussian())
result = model.fit()

result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  price   No. Observations:                 2673
Model:                            GLM   Df Residuals:                     2661
Model Family:                Gaussian   Df Model:                           11
Link Function:               Identity   Scale:                      1.2214e+10
Method:                          IRLS   Log-Likelihood:                -34828.
Date:                Fri, 06 Dec 2024   Deviance:                   3.2501e+13
Time:                        06:30:30   Pearson chi2:                 3.25e+13
No. Iterations:                     3   Pseudo R-squ. (CS):             0.7261
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept           2.942e+04   3.76e+04      0.781      0.435   -4.44e+04    1.03e+05
risk_score             6.6043     93.841      0.070      0.944    -177.320     190.528
population_density    19.5680      1.701     11.507      0.000      16.235      22.901
income                 4.2766      0.161     26.521      0.000       3.961       4.593
white_pct          -2.003e+05   3.77e+04     -5.309      0.000   -2.74e+05   -1.26e+05
black_pct          -2.696e+05   3.87e+04     -6.964      0.000   -3.45e+05   -1.94e+05
asian_pct           3.134e+05    1.1e+05      2.851      0.004    9.79e+04    5.29e+05
hispanic_pct       -1.076e+05   4.06e+04     -2.648      0.008   -1.87e+05   -2.79e+04
ed_pct              4135.1709    348.317     11.872      0.000    3452.483    4817.859
tax_paid_2022         -0.5515      2.308     -0.239      0.811      -5.075       3.972
tax_trend           1881.3697    409.907      4.590      0.000    1077.967    2684.772
unemployment_rate   1.441e+04   1594.215      9.039      0.000    1.13e+04    1.75e+04
======================================================================================
"""